In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

C:\ProgramData\Miniconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [3]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Lambda
from keras.layers import Layer
from keras.layers import Softmax
from keras.layers.convolutional import Conv1D
from keras.layers import MaxPooling1D

import tensorflow as tf
import keras
import keras.backend as K

In [4]:
# Params
maxlen=5
embedding_dim = 100
text_vocabulary_size = 140000
n_epochs = 500

## Load data

In [5]:
w2v =Word2Vec.load('w2v.model')

### Training Events

In [330]:
# Turn extracted events into dataframe
title_list = []
o1_list = []
p_list = []
o2_list = []

# To change to training
with open('./train_events', 'r', encoding='utf8', errors='replace') as f:
    for event in f:
        event = event.split('\t')
        title_list.append(event[12])
        o1_list.append(event[2])
        p_list.append(event[3])
        o2_list.append(event[4])
        

train_df = pd.DataFrame({'title': title_list, 'o1': o1_list, 'p': p_list, 'o2': o2_list})

In [331]:
train_df

,title,o1,p,o2
0,"Hey buddy , can you spare $ 600 for a Google s...",you,spare,$ 600
1,Exxon Mobil offers plan to end Alaska dispute .,Exxon Mobil,offers,plan
2,Exxon Mobil offers plan to end Alaska dispute .,plan,end,Alaska dispute
3,AOL CEO says sales may shrink for two years -p...,AOL CEO,says,sales
4,AOL CEO says sales may shrink for two years -p...,sales,may shrink for,two years -paper
5,Pluspetrol says losing $ 2.4 mln/day in Peru p...,Pluspetrol,says losing,$ 2.4 mln/day
6,EU to urge China to open economy further .,EU,urge,China
7,EU to urge China to open economy further .,China,open,economy
8,"Fed to keep hawkish tone , hold rates steady .",Fed,keep,hawkish tone
9,Weatherford profit jumps 78 percent .,Weatherford profit,jumps,78 percent


In [318]:
train_df_stock

,index,Date,datetime,news_title,source,stock
0,0,2006-10-20 00:00:00-04:00,2006-10-20 16:16:16-04:00,"Inco's Net Soars on Higher Metal Prices, Break...",Bloomberg,1
1,1,2006-10-20 00:00:00-04:00,2006-10-20 16:25:00-04:00,"Hey buddy, can you spare $600 for a Google sha...",Reuters,1
2,2,2006-10-20 00:00:00-04:00,2006-10-20 18:15:00-04:00,Exxon Mobil offers plan to end Alaska dispute.,Reuters,1
3,3,2006-10-20 00:00:00-04:00,2006-10-20 20:08:44-04:00,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",Bloomberg,1
4,4,2006-10-20 00:00:00-04:00,2006-10-21 14:21:00-04:00,AOL CEO says sales may shrink for two years -p...,Reuters,1
5,5,2006-10-20 00:00:00-04:00,2006-10-21 20:11:00-04:00,Pluspetrol says losing $2.4 mln/day in Peru pr...,Reuters,1
6,6,2006-10-20 00:00:00-04:00,2006-10-22 06:46:00-04:00,EU to urge China to open economy further.,Reuters,1
7,7,2006-10-20 00:00:00-04:00,2006-10-22 12:14:00-04:00,"Fed to keep hawkish tone, hold rates steady.",Reuters,1
8,8,2006-10-20 00:00:00-04:00,2006-10-22 20:36:00-04:00,Weatherford profit jumps 78 percent.,Reuters,1
9,9,2006-10-20 00:00:00-04:00,2006-10-22 21:51:00-04:00,Saudi Arabia tells Japan to cut its Nov crude ...,Reuters,1


In [319]:
train_df_stock = pd.read_pickle('train_df.pkl')
train_df = train_df_stock.merge(train_df, how='left', left_on='news_title', right_on='title')

In [334]:
train_df_stock['news_title'][1]

'Hey buddy, can you spare $600 for a Google share?.'

In [333]:
train_df['title'][0]

'Hey buddy , can you spare $ 600 for a Google share ? .'

# Testing Events

In [298]:
# Turn extracted events into dataframe
line_list = []
o1_list = []
p_list = []
o2_list = []

# To change to training
with open('./test_events', 'r', encoding='utf8', errors='replace') as f:
    for event in f:
        event = event.split('\t')
        line_list.append(event[1])
        o1_list.append(event[2])
        p_list.append(event[3])
        o2_list.append(event[4])

test_df = pd.DataFrame({'line_num': line_list, 'o1': o1_list, 'p': p_list, 'o2': o2_list})

In [299]:
test_df['']

,line_num,o1,p,o2
0,1,Private-Equity Managers,Make Moves as,Tax Increases Near
1,15,Battling Rihanna,Puts,Pandora
2,20,Kuwait,Donates,$ 20m
3,25,Packers 38-10,End,NFL Losing Streak
4,28,Park,Leads,Moon
5,31,Argentina,playing last cards in,court battle
6,34,German hint of Greek debt relief,raises,euro hopes
7,36,Psy’s ‘Gangnam Style� � Passes Bieber,Be,Most-Viewed
8,42,Euro Erases Decline Versus Dollar,Is,Little
9,42,Little,Changed at,$ 1.2980


In [291]:
test_df.head()

,line_num,o1,p,o2
0,1,Private-Equity Managers,Make Moves as,Tax Increases Near
1,15,Battling Rihanna,Puts,Pandora
2,20,Kuwait,Donates,$ 20m
3,25,Packers 38-10,End,NFL Losing Streak
4,28,Park,Leads,Moon


In [293]:
test_df['line_num'] = test_df['line_num'].astype(int) + 228214

In [300]:
test_df[test_df['o1'] == 'Bankruptcy']

,line_num,o1,p,o2
23122,60001,Bankruptcy,Seek,Sale
27454,70543,Bankruptcy,Sell,Assets
28981,74303,Bankruptcy,Losing,Wal-Mart Business
31726,81156,Bankruptcy,Sell,Assets


In [294]:
test_df.head()

,line_num,o1,p,o2
0,228215,Private-Equity Managers,Make Moves as,Tax Increases Near
1,228229,Battling Rihanna,Puts,Pandora
2,228234,Kuwait,Donates,$ 20m
3,228239,Packers 38-10,End,NFL Losing Streak
4,228242,Park,Leads,Moon


In [296]:
test_df_stock.merge(test_df, how='left', left_on='index', right_on='line_num')

,index,Date,datetime,news_title,source,stock,line_num,o1,p,o2
0,288215,2012-11-26 00:00:00-05:00,2012-11-26 00:00:01-05:00,Private-Equity Managers Make Moves as Tax Incr...,Bloomberg,0,288215.0,Bankruptcy,Seek,Sale
1,288216,2012-11-26 00:00:00-05:00,2012-11-26 00:00:01-05:00,Canada Opposition Seeks Upset in Calgary Conse...,Bloomberg,0,288216.0,U.S. auto sales growth,slows in,April
2,288217,2012-11-26 00:00:00-05:00,2012-11-26 00:00:01-05:00,Cohen’s SAC Faces Client Queries as Investigat...,Bloomberg,0,NaN,NaN,NaN,NaN
3,288218,2012-11-26 00:00:00-05:00,2012-11-26 00:00:01-05:00,Kansas Shows What’s the Matter With GOP Immigr...,Bloomberg,0,288218.0,Viacom and CBS,beat expectations in,tough TV ad market
4,288219,2012-11-26 00:00:00-05:00,2012-11-26 00:00:09-05:00,"Notre Dame Remains No. 1 in BCS, Awaits Title ...",Bloomberg,0,288219.0,Bayern Routs Barca,Set Up,All-German Champions League Final
5,288220,2012-11-26 00:00:00-05:00,2012-11-26 00:00:15-05:00,Bounce Houses Boost Child Injuries 15-Fold in ...,Bloomberg,0,288220.0,Syncreon Logistics Business,Said to Be for,Sale
6,288221,2012-11-26 00:00:00-05:00,2012-11-26 00:00:30-05:00,Rockets Mourn Passing of Coach McHale’s 23-Yea...,Bloomberg,0,NaN,NaN,NaN,NaN
7,288222,2012-11-26 00:00:00-05:00,2012-11-26 00:00:32-05:00,Grinnell’s Taylor Scores 21 After Record-Break...,Bloomberg,0,NaN,NaN,NaN,NaN
8,288223,2012-11-26 00:00:00-05:00,2012-11-26 00:00:36-05:00,"Auburn’s Chizik Among 5 Coaches Fired, Two Yea...",Bloomberg,0,288223.0,husband,sentenced in,trade secret theft case
9,288224,2012-11-26 00:00:00-05:00,2012-11-26 00:01:00-05:00,Chrysler Unveils Pricier Town & Country at Aut...,Bloomberg,0,NaN,NaN,NaN,NaN


In [288]:
test_df_stock = pd.read_pickle('test_df.pkl')
test_df['line_num'] = test_df['line_num'].astype(int) + 228214
test_df = test_df_stock.merge(test_df, how='left', left_on='index', right_on='line_num')

In [289]:
test_df

,index,Date,datetime,news_title,source,stock,line_num,o1,p,o2
0,288215,2012-11-26 00:00:00-05:00,2012-11-26 00:00:01-05:00,Private-Equity Managers Make Moves as Tax Incr...,Bloomberg,0,288215.0,Bankruptcy,Seek,Sale
1,288216,2012-11-26 00:00:00-05:00,2012-11-26 00:00:01-05:00,Canada Opposition Seeks Upset in Calgary Conse...,Bloomberg,0,288216.0,U.S. auto sales growth,slows in,April
2,288217,2012-11-26 00:00:00-05:00,2012-11-26 00:00:01-05:00,Cohen’s SAC Faces Client Queries as Investigat...,Bloomberg,0,NaN,NaN,NaN,NaN
3,288218,2012-11-26 00:00:00-05:00,2012-11-26 00:00:01-05:00,Kansas Shows What’s the Matter With GOP Immigr...,Bloomberg,0,288218.0,Viacom and CBS,beat expectations in,tough TV ad market
4,288219,2012-11-26 00:00:00-05:00,2012-11-26 00:00:09-05:00,"Notre Dame Remains No. 1 in BCS, Awaits Title ...",Bloomberg,0,288219.0,Bayern Routs Barca,Set Up,All-German Champions League Final
5,288220,2012-11-26 00:00:00-05:00,2012-11-26 00:00:15-05:00,Bounce Houses Boost Child Injuries 15-Fold in ...,Bloomberg,0,288220.0,Syncreon Logistics Business,Said to Be for,Sale
6,288221,2012-11-26 00:00:00-05:00,2012-11-26 00:00:30-05:00,Rockets Mourn Passing of Coach McHale’s 23-Yea...,Bloomberg,0,NaN,NaN,NaN,NaN
7,288222,2012-11-26 00:00:00-05:00,2012-11-26 00:00:32-05:00,Grinnell’s Taylor Scores 21 After Record-Break...,Bloomberg,0,NaN,NaN,NaN,NaN
8,288223,2012-11-26 00:00:00-05:00,2012-11-26 00:00:36-05:00,"Auburn’s Chizik Among 5 Coaches Fired, Two Yea...",Bloomberg,0,288223.0,husband,sentenced in,trade secret theft case
9,288224,2012-11-26 00:00:00-05:00,2012-11-26 00:01:00-05:00,Chrysler Unveils Pricier Town & Country at Aut...,Bloomberg,0,NaN,NaN,NaN,NaN


## Associate word_indices with embeddings

### Create weight matrix

In [247]:
# From https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
embedding_matrix = np.zeros((text_vocabulary_size, 100))

In [248]:
#create a weight matrix for words in training docs
for word, i in t.word_index.items():
    try:
        embedding_vector = w2v.wv.get_vector(word)
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue

### Train

In [249]:
train_df.loc[:, ['o1', 'p', 'o2']] = train_df.loc[:, ['o1', 'p', 'o2']].fillna('')

In [250]:
# Preprocess
train_df['o1'] = train_df.o1.map(simple_preprocess)
train_df['p'] = train_df.p.map(simple_preprocess)
train_df['o2'] = train_df.o2.map(simple_preprocess)

In [251]:
# Tokenize and fit on text
t = Tokenizer()
t.fit_on_texts(train_df['o1'] + train_df['p'] + train_df['o2'])

In [252]:
# Text to sequence
train_df['input_o1'] = t.texts_to_sequences(train_df['o1'])
train_df['input_p'] = t.texts_to_sequences(train_df['p'])
train_df['input_o2'] = t.texts_to_sequences(train_df['o2'])

In [253]:
# Pad sequences
input_o1 = pad_sequences(train_df['input_o1'], maxlen=maxlen, padding='post')
input_p = pad_sequences(train_df['input_p'], maxlen=maxlen, padding='post')
input_o2 = pad_sequences(train_df['input_o2'], maxlen=maxlen, padding='post')

In [254]:
train_df['input_o1'] = [np.squeeze(x) for x in np.split(input_o1, input_o1.shape[0])]
train_df['input_p'] = [np.squeeze(x) for x in np.split(input_p, input_o1.shape[0])]
train_df['input_o2'] = [np.squeeze(x) for x in np.split(input_o2, input_o1.shape[0])]

### Test

In [255]:
test_df.loc[:, ['o1', 'p', 'o2']] = test_df.loc[:, ['o1', 'p', 'o2']].fillna('')

In [256]:
# Preprocess
test_df['o1'] = test_df.o1.map(simple_preprocess)
test_df['p'] = test_df.p.map(simple_preprocess)
test_df['o2'] = test_df.o2.map(simple_preprocess)

In [257]:
# Tokenize and fit on text
t = Tokenizer()
t.fit_on_texts(test_df['o1'] + test_df['p'] + test_df['o2'])

In [258]:
# Text to sequence
test_df['input_o1'] = t.texts_to_sequences(test_df['o1'])
test_df['input_p'] = t.texts_to_sequences(test_df['p'])
test_df['input_o2'] = t.texts_to_sequences(test_df['o2'])

In [259]:
# Pad sequences
input_o1 = pad_sequences(test_df['input_o1'], maxlen=maxlen, padding='post')
input_p = pad_sequences(test_df['input_p'], maxlen=maxlen, padding='post')
input_o2 = pad_sequences(test_df['input_o2'], maxlen=maxlen, padding='post')

In [260]:
test_df['input_o1'] = [np.squeeze(x) for x in np.split(input_o1, input_o1.shape[0])]
test_df['input_p'] = [np.squeeze(x) for x in np.split(input_p, input_o1.shape[0])]
test_df['input_o2'] = [np.squeeze(x) for x in np.split(input_o2, input_o1.shape[0])]

In [261]:
test_df['input_o1'] = [embedding_matrix[x] for x in test_df['input_o1']]
test_df['input_p'] = [embedding_matrix[x] for x in test_df['input_p']]
test_df['input_o2'] = [embedding_matrix[x] for x in test_df['input_o2']]

### Aggregate event embeddings by event

In [262]:
train_df['stock'] = train_df['stock'].astype(int)

g = train_df.groupby(['Date', 'line_num'])

train_df = pd.concat([g.input_o1.apply(np.mean, axis=0),
                    g.input_p.apply(np.mean, axis=0),
                    g.input_o2.apply(np.mean, axis=0),
                    g.stock.apply(np.mean, axis=0)                    
                   ],
                   axis=1)

train_df.reset_index(inplace=True)

In [263]:
test_df['stock'] = test_df['stock'].astype(int)

g = test_df.groupby(['Date', 'line_num'])

test_df = pd.concat([g.input_o1.apply(np.mean, axis=0),
                    g.input_p.apply(np.mean, axis=0),
                    g.input_o2.apply(np.mean, axis=0),
                    g.stock.apply(np.mean, axis=0)                    
                   ],
                   axis=1)

test_df.reset_index(inplace=True)

### Aggregate events by day

In [264]:
g = train_df.groupby('Date')

agg_df = pd.concat([g.input_o1.apply(np.mean, axis=0),
                    g.input_p.apply(np.mean, axis=0),
                    g.input_o2.apply(np.mean, axis=0),
                    g.stock.apply(np.mean, axis=0)
                   ],
                   axis=1)

train_df.reset_index(inplace=True)

train_df.to_pickle('agg_train_df.pkl')

In [267]:
test_df

,index,input_o1,input_p,input_o2,stock


In [265]:
g = test_df.groupby('Date')

test_df = pd.concat([g.input_o1.apply(np.mean, axis=0),
                    g.input_p.apply(np.mean, axis=0),
                    g.input_o2.apply(np.mean, axis=0),
                    g.stock.apply(np.mean, axis=0)
                   ],
                   axis=1)

test_df.reset_index(inplace=True)

test_df.to_pickle('agg_test_df.pkl')

KeyError: 'Date'

## Load data

In [ ]:
train_df = pd.read_pickle('agg_train_df.pkl')

In [ ]:
test_df = pd.read_pickle('agg_train_df.pkl')

In [217]:
# Convert back from series of arrays to large array of (events, maxlen, embedding_dims)

input_o1 = np.array(agg_df.input_o1.to_list())
input_p = np.array(agg_df.input_p.to_list())
input_o2 = np.array(agg_df.input_o2.to_list())

y_train = np.array(agg_df.stock)

## Stock Prediction Model

In [218]:
seqlen = input_o1.shape[0]

In [219]:
from keras.regularizers import l2

In [220]:
l2_penalty = 0.005

In [228]:
K.clear_session()

In [229]:
# Embed events
# Take training events and return embedded vectors
# Create copy of training events, randomly sample events from data, return embedded vectors
# Multiply each by tensor
# Calculate loss


# Take real inputs
seqlen = input_o1.shape[0]
input_o1_ = Input(shape=(seqlen, maxlen, embedding_dim), dtype='float32', name='o1')
input_p_ = Input(shape=(seqlen, maxlen, embedding_dim), dtype='float32', name='p')
input_o2_ = Input(shape=(seqlen, maxlen, embedding_dim), dtype='float32', name='o2')

# # Average embeddings for o1, p, o2
average = keras.layers.Lambda(lambda x: keras.backend.mean(x, axis=2))
o1_ = average(input_o1_) # Output dim (100)
p_ = average(input_p_)
o2_ = average(input_o2_)

# Concatenate_event parts
event_ = keras.layers.Concatenate(axis=-1)([o1_, p_, o2_])

event_day_ = Conv1D(filters=embedding_dim, kernel_size=1, strides=1, padding='same', activation='relu',
                    use_bias=True, kernel_regularizer=l2(l2_penalty), bias_regularizer=l2(l2_penalty)
                   )(event_)

event_week_ = Conv1D(filters=embedding_dim, kernel_size=5, strides=1, padding='causal', activation='relu',
                     use_bias=True, kernel_regularizer=l2(l2_penalty), bias_regularizer=l2(l2_penalty)
                    )(event_)

event_month_ = Conv1D(filters=embedding_dim, kernel_size=20, strides=1, padding='causal', activation='relu',
                      use_bias=True, kernel_regularizer=l2(l2_penalty), bias_regularizer=l2(l2_penalty)
                     )(event_)


# # Max pooling of weekly and monthly events
# # Narrow convolution of 3 neighbouring
# max_pool_week_ = MaxPooling1D(pool_size=5, strides=None, padding='valid', data_format='channels_last'
#                              )(event_week_)

# max_pool_month_ = MaxPooling1D(pool_size=3, strides=None, padding='valid', data_format='channels_last'
#                               )(event_month_)

# # Concatenate daily, weekly, monthly
concat_ = keras.layers.Concatenate(axis=2)([event_day_, event_week_, event_month_])

# Hidden layer
hidden_ = keras.layers.Dense(units=50, activation='relu', use_bias=True, kernel_initializer='glorot_uniform', 
                             bias_initializer='zeros', kernel_regularizer=l2(l2_penalty), 
                             bias_regularizer=l2(l2_penalty),
                            )(concat_)

# Softmax layer
y_pred_ = keras.layers.Dense(units=1, activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform', 
                             bias_initializer='zeros', kernel_regularizer=l2(l2_penalty), 
                             bias_regularizer=l2(l2_penalty),
                            )(hidden_)

model = Model(inputs=[input_o1_, input_p_, input_o2_], outputs=y_pred_)


In [230]:
from keras import optimizers

In [231]:
sgd = optimizers.SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)

In [232]:
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [233]:
model.fit(x=[np.expand_dims(input_o1, 0), 
             np.expand_dims(input_p, 0), 
             np.expand_dims(input_o2, 0)],
          y=np.expand_dims(np.expand_dims(y_train, -1), 0), 
          batch_size=1, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 568ms/step - loss: 3.3816 - acc: 0.5422
Epoch 2/100
1/1 [==============================] - 0s 35ms/step - loss: 3.3815 - acc: 0.5409
Epoch 3/100
1/1 [==============================] - 0s 20ms/step - loss: 3.3814 - acc: 0.5429
Epoch 4/100
1/1 [==============================] - 0s 22ms/step - loss: 3.3813 - acc: 0.5455
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 3.3812 - acc: 0.5488
Epoch 6/100
1/1 [==============================] - 0s 21ms/step - loss: 3.3810 - acc: 0.5455
Epoch 7/100
1/1 [==============================] - 0s 21ms/step - loss: 3.3808 - acc: 0.5442
Epoch 8/100
1/1 [==============================] - 0s 20ms/step - loss: 3.3806 - acc: 0.5422
Epoch 9/100
1/1 [==============================] - 0s 20ms/step - loss: 3.3804 - acc: 0.5429
Epoch 10/100
1/1 [==============================] - 0s 22ms/step - loss: 3.3802 - acc: 0.5403
Epoch 11/100
1/1 [==============================] - 0s 21ms/step - l

Epoch 89/100
1/1 [==============================] - 0s 22ms/step - loss: 3.3573 - acc: 0.5475
Epoch 90/100
1/1 [==============================] - 0s 23ms/step - loss: 3.3570 - acc: 0.5475
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 3.3568 - acc: 0.5475
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 3.3565 - acc: 0.5475
Epoch 93/100
1/1 [==============================] - 0s 22ms/step - loss: 3.3562 - acc: 0.5475
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 3.3559 - acc: 0.5475
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 3.3556 - acc: 0.5475
Epoch 96/100
1/1 [==============================] - 0s 21ms/step - loss: 3.3554 - acc: 0.5475
Epoch 97/100
1/1 [==============================] - 0s 20ms/step - loss: 3.3551 - acc: 0.5475
Epoch 98/100
1/1 [==============================] - 0s 21ms/step - loss: 3.3548 - acc: 0.5475
Epoch 99/100
1/1 [==============================] - 0s 21ms/

In [177]:
np.mean(y_train)

0.547478716437459